![HPE DEV Logo](Pictures/hpe-dev-logo.png "HPE DEV Hack Shack")

# Hack Shack

Powered by [HPE DEV Team](hpedev.io)

### Speakers : Troy Heber, François Donzé

### Redfish overview workshop

Version 0.41

<img src="https://redfish.dmtf.org/sites/default/files/DMTF_Redfish_logo_R.jpg" alt="Redfish Logo" style="width: 125px;"/> 

# Using unique code to retrieve properties from different Redfish implementations

## Introduction

This notebook contains a single Python program, explained and executed step by step. The goal of this lab is to retrieve the BMC MAC addresses from an OpenBMC and from an HPE iLO 5 using a single piece of code.

Please note that the account on your dedicated OpenBMC simulator has Administrator privileges, while you have Read-Only permissions on the physical shared iLO 5.

For simplicity and didactic reasons, the following code is not optimized and does not handle errors or test return codes. The simple syntax used should be easy to understand by non-Python knowledgeable students.

### Setting the scene

The following cell imports the minimum required Python modules and defines **global variables** (credentials and IP addresses). The `redfish` module has been installed on the Jupyter server using the `pip3 install redfish` command as explained in the [DMTF's Python Redfish library](https://github.com/DMTF/python-redfish-library) GitHub site.

In [1]:
# -*- coding: utf-8 -*-

import os
import re
import sys
import json
import redfish # Module sources at: https://github.com/DMTF/python-redfish-library/blob/master/src/redfish/rest/v1.py

###########################################################################################################
#
# Variables
#
###########################################################################################################

# Set Student ID number
Stud="00"
print ("You are Student" + Stud + "\n\n")

# OpenBMC and iLO common credentials
user="student"
Password = "P@ssw0rd!"

# OpenBMC and iLO IP addresses with port
ObmcIP = "openbmcs:443" + Stud
iLO5IP = "ilo5"

You are Student00




## Redfish session creation and Root service content

The following Python cell creates a Redfish session and saves the Root service content (`/redfish/v1`) in a variable called `service_root`. 

Session details (Session Key, Token, Location...) are stored in the  **`_redfishobj`** Python object. This `_redfishobj` object is automatically and silently removed at the end of the cell and the corresponding session is automatically deleted in the remote BMC.

Hence, each Jupyter cell in this notebook starts with the creation of a new session object. This may not be optimal in real life, but allows for an easy comparison of several Redfish implementations in a single notebook.

In [2]:
IP = ObmcIP
base_url = "https://" + IP

###########################################################################################################
#
# Redfish session creation and root services listing
#
###########################################################################################################
if __name__ == "__main__":
    # Set up the Redfish session object (_redfishobj)
    with redfish.redfish_client( base_url = base_url, username = user, password = Password ) as _redfishobj:      
        
        SessionKey = _redfishobj.get_session_key()
        SessionLocation = _redfishobj.get_session_location()
        print ("Session Key (Token): " + SessionKey + "\n" +
            "Session Location   : " + SessionLocation + "\n")
        
        # Retrieve and print the Redfish Root service content from the standard location /redfish/v1/
        service_root = _redfishobj.get( "/redfish/v1/" )
        print ("Root Services:" + "\n" + json.dumps( service_root.dict, indent=4 ) + "\n")

Session Key (Token): Y0qMeRgWMkq0BNJJBi1l
Session Location   : /redfish/v1/SessionService/Sessions/gjbqNcB1hT

Root Services:
{
    "@odata.id": "/redfish/v1",
    "@odata.type": "#ServiceRoot.v1_5_0.ServiceRoot",
    "AccountService": {
        "@odata.id": "/redfish/v1/AccountService"
    },
    "CertificateService": {
        "@odata.id": "/redfish/v1/CertificateService"
    },
    "Chassis": {
        "@odata.id": "/redfish/v1/Chassis"
    },
    "Id": "RootService",
    "JsonSchemas": {
        "@odata.id": "/redfish/v1/JsonSchemas"
    },
    "Links": {
        "Sessions": {
            "@odata.id": "/redfish/v1/SessionService/Sessions"
        }
    },
    "Managers": {
        "@odata.id": "/redfish/v1/Managers"
    },
    "Name": "Root Service",
    "RedfishVersion": "1.9.0",
    "Registries": {
        "@odata.id": "/redfish/v1/Registries"
    },
    "SessionService": {
        "@odata.id": "/redfish/v1/SessionService"
    },
    "Systems": {
        "@odata.id": "/redfish/v1

## Managers collection listing

### OpenBMC

As explained in the previous notebook, the standard Redfish location of a server BMC(s) is found at `/redfish/v1/Managers/{item}`, where `{item}` varies from one Redfish implementation to another. 

The next cell retrieves and prints the `Managers` collection from an OpenBMC simulator. This collection has only one item named `bmc`. Run it.

In [3]:
IP = ObmcIP
base_url = "https://" + IP
###########################################################################################################
#
# Managers service listing
#
###########################################################################################################
if __name__ == "__main__":
    with redfish.redfish_client( base_url = base_url, username = user, password = Password ) as _redfishobj:
        service_root = _redfishobj.get( "/redfish/v1/" )
        
        # Retrieve the location of the "Managers" service and its content.
        # To be perfect, we should verify that the `Managers` resource is present in the ServiceRoot schema
        manager_collection = _redfishobj.get( service_root.dict["Managers"]["@odata.id"] )
        print ( "Managers collection: " + "\n" + json.dumps( manager_collection.dict, indent=4 ) + "\n" )

Managers collection: 
{
    "@odata.id": "/redfish/v1/Managers",
    "@odata.type": "#ManagerCollection.ManagerCollection",
    "Members": [
        {
            "@odata.id": "/redfish/v1/Managers/bmc"
        }
    ],
    "Members@odata.count": 1,
    "Name": "Manager Collection"
}



### HPE iLO 5

The following program retrieves the `Managers` collection of an **HPE ProLiant iLO 5**.

In this example, the HPE server has only one item called `1`. Other HPE servers may have several BMCs called differently.

**CONCLUSION**: If you want to your Redfish programs to work against different Redfish implementations, it is mandatory to avoid assumptions on the URIs of services. You must crawl the Redfish tree and mechanically discover the desired URIs.

In [4]:
IP = iLO5IP
base_url = "https://" + IP
###########################################################################################################
#
# Managers service listing
#
###########################################################################################################
if __name__ == "__main__":
    with redfish.redfish_client( base_url = base_url, username = user, password = Password ) as _redfishobj:
        service_root = _redfishobj.get( "/redfish/v1/" )
        # Retrieve the location of the "Managers" service and its content.
        # To be perfect, we should verify that the `Managers` resource is present in the ServiceRoot schema
        manager_collection = _redfishobj.get( service_root.dict["Managers"]["@odata.id"] )
        print ( "Managers location: " + "\n" + json.dumps( manager_collection.dict, indent=4 ) + "\n" )

Managers location: 
{
    "@odata.context": "/redfish/v1/$metadata#ManagerCollection.ManagerCollection",
    "@odata.etag": "W/\"AA6D42B0\"",
    "@odata.id": "/redfish/v1/Managers/",
    "@odata.type": "#ManagerCollection.ManagerCollection",
    "Description": "Managers view",
    "Name": "Managers",
    "Members": [
        {
            "@odata.id": "/redfish/v1/Managers/1/"
        }
    ],
    "Members@odata.count": 1
}



## BMC properties and `EthernetInterfaces` collection

The next two Python cells use the exact same code to locate the BMC properties of an HPE iLO 5 first and then an OpenBMC simulator. Then, they locate and print the collection of `EthernetInterfaces` found in each BMC.

The logic of this code starts at the Root Redfish tree and then follows the desired links. The main steps are; 1) Create a Redfish session object `_redfishobj`; 2) Retrieve and save the Root service content in variable `service_root`; 3) Retrieve and save the URI of the `Managers` collection in variable `manager_collection`; 4) For each BMC URI, save its properties in variable `manager_resources`; and 5) Print  those properties as well as the collection of `EthernetInterfaces`


### OpenBMC 

The OpenBMC simulator returns **two** BMC NICs under `/redfish/v1/Managers/bmc/EthernetInterfaces`. 

In [5]:
IP = ObmcIP
base_url = "https://" + IP
###########################################################################################################
#
# BMC properties and Ethernet Network Interfaces listing
#
###########################################################################################################
if __name__ == "__main__":
    with redfish.redfish_client( base_url = base_url, username = user, password = Password ) as _redfishobj:  
        service_root = _redfishobj.get( "/redfish/v1/" )
        manager_collection = _redfishobj.get( service_root.dict["Managers"]["@odata.id"] )
    
        for manager_member in manager_collection.dict["Members"]:
            manager_resources = _redfishobj.get( manager_member["@odata.id"] )
            print ( "Manager Resources" + "\n" + json.dumps(manager_resources.dict, indent=4) + "\n" )  
            print ("##############################################################################" + "\n")
            ethernet_network_interface_collection = _redfishobj.get(manager_resources.dict["EthernetInterfaces"]["@odata.id"])
            print ( "Ethernet Network Interface Collection" + "\n" + 
                   json.dumps(ethernet_network_interface_collection.dict, indent=4) + "\n")   

Manager Resources
{
    "@odata.id": "/redfish/v1/Managers/bmc",
    "@odata.type": "#Manager.v1_3_0.Manager",
    "Actions": {
        "#Manager.Reset": {
            "ResetType@Redfish.AllowableValues": [
                "GracefulRestart"
            ],
            "target": "/redfish/v1/Managers/bmc/Actions/Manager.Reset"
        }
    },
    "DateTime": "2020-08-24T16:37:51+00:00",
    "Description": "Baseboard Management Controller",
    "EthernetInterfaces": {
        "@odata.id": "/redfish/v1/Managers/bmc/EthernetInterfaces"
    },
    "FirmwareVersion": "2.8.0-dev-1427-g64e281927",
    "GraphicalConsole": {
        "ConnectTypesSupported": [
            "KVMIP"
        ],
        "MaxConcurrentSessions": 4,
        "ServiceEnabled": true
    },
    "Id": "bmc",
    "Links": {
        "ManagerForChassis": [
            {
                "@odata.id": "/redfish/v1/Chassis/chassis"
            }
        ],
        "ManagerForChassis@odata.count": 1,
        "ManagerForServers": [
 

### HPE iLO 5

The HPE iLO 5 returns **three** BMC NICs under `/redifsh/v1/Managers/1/EthernetInterfaces`. 

In [6]:
IP = iLO5IP
base_url = "https://" + IP
###########################################################################################################
#
# BMC properties and Ethernet Network Interfaces listing
#
###########################################################################################################
if __name__ == "__main__":
    with redfish.redfish_client( base_url = base_url, username = user, password = Password ) as _redfishobj:  
        service_root = _redfishobj.get( "/redfish/v1/" )
        manager_collection = _redfishobj.get( service_root.dict["Managers"]["@odata.id"] )
    
        for manager_member in manager_collection.dict["Members"]:
            manager_resources = _redfishobj.get( manager_member["@odata.id"] )
            print ( "Manager Resources" + "\n" + json.dumps(manager_resources.dict, indent=4) + "\n" )  
            print ("##############################################################################" + "\n")
            ethernet_network_interface_collection = _redfishobj.get(manager_resources.dict["EthernetInterfaces"]["@odata.id"])
            print ( "Ethernet Network Interface Collection" + "\n" + 
                   json.dumps(ethernet_network_interface_collection.dict, indent=4) + "\n")   

Manager Resources
{
    "@odata.context": "/redfish/v1/$metadata#Manager.Manager",
    "@odata.etag": "W/\"120FCADD\"",
    "@odata.id": "/redfish/v1/Managers/1/",
    "@odata.type": "#Manager.v1_3_3.Manager",
    "Id": "1",
    "Actions": {
        "#Manager.Reset": {
            "target": "/redfish/v1/Managers/1/Actions/Manager.Reset/"
        }
    },
    "CommandShell": {
        "ConnectTypesSupported": [
            "SSH",
            "Oem"
        ],
        "MaxConcurrentSessions": 9,
        "ServiceEnabled": true
    },
    "EthernetInterfaces": {
        "@odata.id": "/redfish/v1/Managers/1/EthernetInterfaces/"
    },
    "FirmwareVersion": "iLO 5 v1.43",
    "GraphicalConsole": {
        "ConnectTypesSupported": [
            "KVMIP"
        ],
        "MaxConcurrentSessions": 10,
        "ServiceEnabled": true
    },
    "HostInterfaces": {
        "@odata.id": "/redfish/v1/Managers/1/HostInterfaces/"
    },
    "Links": {
        "ManagerInChassis": {
            "@odata.

## BMC NICs MAC addresses

Using the exact same code again, we extract the MAC addresses of the two OpenBMC NICs and then the three iLO 5 NICs, regardless their different names and locations.

### OpenBMC

In [7]:
IP = ObmcIP
base_url = "https://" + IP
###########################################################################################################
#
# BMC NICs MAC addresses extraction
#
###########################################################################################################
if __name__ == "__main__":
    with redfish.redfish_client( base_url = base_url, username = user, password = Password ) as _redfishobj:  
        # Retrieve the Redfish root services from the standard location /redfish/v1/
        service_root = _redfishobj.get( "/redfish/v1/" )
        manager_collection = _redfishobj.get( service_root.dict["Managers"]["@odata.id"] )

        for manager_member in manager_collection.dict["Members"]:
            manager_resources = _redfishobj.get( manager_member["@odata.id"] )
            ethernet_network_interface_collection = _redfishobj.get(manager_resources.dict["EthernetInterfaces"]["@odata.id"])
            
            # For each ethernet network interface, get its properties and print the location, Id and MAC Address
            for ethernet_network_interface in ethernet_network_interface_collection.dict["Members"]:
                print (json.dumps(ethernet_network_interface))
                ethernet_network_interface_resources = _redfishobj.get(ethernet_network_interface["@odata.id"])
                
                print (json.dumps(ethernet_network_interface_resources.dict["Id"], indent=4) + ":\t" +
                       json.dumps(ethernet_network_interface_resources.dict["MACAddress"], indent=4) + "\n")   

{"@odata.id": "/redfish/v1/Managers/bmc/EthernetInterfaces/eth0"}
"eth0":	"c0:ff:ee:00:00:01"

{"@odata.id": "/redfish/v1/Managers/bmc/EthernetInterfaces/sit0"}
"sit0":	"00:00:00:00:00:00"



### HPE iLO 5


In [8]:
IP = iLO5IP
base_url = "https://" + IP

###########################################################################################################
#
# BMC NICs MAC addresses extraction
#
###########################################################################################################
if __name__ == "__main__":
    with redfish.redfish_client( base_url = base_url, username = user, password = Password ) as _redfishobj:  
        # Retrieve the Redfish root services from the standard location /redfish/v1/
        service_root = _redfishobj.get( "/redfish/v1/" )
        manager_collection = _redfishobj.get( service_root.dict["Managers"]["@odata.id"] )

        for manager_member in manager_collection.dict["Members"]:
            manager_resources = _redfishobj.get( manager_member["@odata.id"] )
            ethernet_network_interface_collection = _redfishobj.get(manager_resources.dict["EthernetInterfaces"]["@odata.id"])
            
            # For each ethernet network interface, get its properties and print the location, Id and MAC Address
            for ethernet_network_interface in ethernet_network_interface_collection.dict["Members"]:
                print (json.dumps(ethernet_network_interface))
                ethernet_network_interface_resources = _redfishobj.get(ethernet_network_interface["@odata.id"])
                
                print (json.dumps(ethernet_network_interface_resources.dict["Id"], indent=4) + ":\t" +
                       json.dumps(ethernet_network_interface_resources.dict["MACAddress"], indent=4) + "\n")  

{"@odata.id": "/redfish/v1/Managers/1/EthernetInterfaces/1/"}
"1":	"08:F1:EA:73:5E:28"

{"@odata.id": "/redfish/v1/Managers/1/EthernetInterfaces/2/"}
"2":	"08:F1:EA:73:5E:29"

{"@odata.id": "/redfish/v1/Managers/1/EthernetInterfaces/3/"}
"3":	"00:CA:FE:F0:0D:04"



## Congratulations !

You may now continue to the **[Conclusion/Survey](4-Conclusion-Survey.ipynb)** notebook. Thanks you for participating to this lab.